In [3]:
# from qiskit.opflow import I, Z, X, Y, SparsePauliOp
import numpy as np
# from juqcs import Juqcs
# 假设有 3 台机器和 3 个操作
m = 3 # 机器数量
n = 2  # 操作数量
l = 1  # 用于表示 C_max 的二进制位数

# 处理时间矩阵 (例如)
p = np.array([[1, 2, 3],[3, 1, 2]])

# 常数
K1 = 20
P = 20

# 量子比特数量
num_qubits = n * m + l

In [4]:
#
def pauli_z_term(index, num_qubits):
    """初始化一个 Pauli-I 门的列表,指定索引 index 替换为 Pauli-Z 门"""
    paulis = ['I'] * num_qubits
    paulis[index] = 'Z'
    return ''.join(paulis)  # 将列表转化为字符串

def pauli_x_term(index, num_qubits):
    paulis = ['I'] * num_qubits
    paulis[index] = 'X'
    return ''.join(paulis)


In [5]:
from qiskit.quantum_info import Pauli, SparsePauliOp, PauliList

# 常数项
# constant_term = 1/2 * sum(2**h for h in range(l))
pauli_strings = []
cofficients = []
# 构建 Pauli-Z 项
z_terms = []
for h in range(l):
    coef = - 2**h / 2
    z_term = pauli_z_term(m * n + h, num_qubits)
    z_terms.append((coef, z_term))

# 从z_terms中提取Pauli字符串和系数
pauli_strings = [term[1] for term in z_terms]
coefficients = [term[0] for term in z_terms]
# 转换为 SparsePauliOp
paulis = [Pauli(label) for label in pauli_strings]
hamiltonian1 = SparsePauliOp.from_list(list(zip(pauli_strings, coefficients)))
print(hamiltonian1)

SparsePauliOp(['IIIIIIZ'],
              coeffs=[-0.5+0.j])


In [6]:
pauli_strings = []
cofficients = []
# 常数项
# constant_term1 = K1 * n * ((m / 2) - 1)**2
# pauli_strings.append('I'*num_qubits)
# cofficients.append(constant_term1)

# 线性项
x_terms = []
# coef_linear = -1 * K1 * (m / 2 - 1)   `
for i in range(n):
    for j in range(m):
        coef = -1 * K1 * (m - 1)/2
        x_term = pauli_z_term(i * m + j, num_qubits)
        pauli_strings.append(x_term)
        cofficients.append(coef)
        # linear_terms.append((coef, x_term))

# 二次项
# quadratic_terms = []
# coef_quadratic = K1 / 4
for i in range(n):
    coef = K1 / 4
    for j1 in range(m):
        for j2 in range(m):
            if j1 != j2:
                x_term1 = pauli_z_term(i * m + j1, num_qubits)
                x_term2 = pauli_z_term(i * m + j2, num_qubits)
                combined_op = ''.join(['I' if c1 == 'I' and c2 == 'I' else 'Z' if c1 == 'Z' or c2 == 'Z' else 'I'
                                           for c1, c2 in zip(x_term1, x_term2)])
                pauli_strings.append(combined_op)
                cofficients.append(coef)
            # quadratic_terms.append((coef, x_term1 + x_term2)) # 二次项的具体形式应该是每对 Zij 和 Zik 的组合

# 组合所有项
# pauli_strings = [term[1] for term in linear_terms + quadratic_terms]
# coefficients = [term[0] for term in linear_terms + quadratic_terms]
print(pauli_strings)
print(cofficients)
# 转换为 SparsePauliOp
paulis = [Pauli(label) for label in pauli_strings]
hamiltonian2 = SparsePauliOp.from_list(list(zip(pauli_strings, cofficients)))

print(hamiltonian2)

['ZIIIIII', 'IZIIIII', 'IIZIIII', 'IIIZIII', 'IIIIZII', 'IIIIIZI', 'ZZIIIII', 'ZIZIIII', 'ZZIIIII', 'IZZIIII', 'ZIZIIII', 'IZZIIII', 'IIIZZII', 'IIIZIZI', 'IIIZZII', 'IIIIZZI', 'IIIZIZI', 'IIIIZZI']
[-20.0, -20.0, -20.0, -20.0, -20.0, -20.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]
SparsePauliOp(['ZIIIIII', 'IZIIIII', 'IIZIIII', 'IIIZIII', 'IIIIZII', 'IIIIIZI', 'ZZIIIII', 'ZIZIIII', 'ZZIIIII', 'IZZIIII', 'ZIZIIII', 'IZZIIII', 'IIIZZII', 'IIIZIZI', 'IIIZZII', 'IIIIZZI', 'IIIZIZI', 'IIIIZZI'],
              coeffs=[-20.+0.j, -20.+0.j, -20.+0.j, -20.+0.j, -20.+0.j, -20.+0.j,   5.+0.j,
   5.+0.j,   5.+0.j,   5.+0.j,   5.+0.j,   5.+0.j,   5.+0.j,   5.+0.j,
   5.+0.j,   5.+0.j,   5.+0.j,   5.+0.j])


In [5]:
# 初始化数组
pauli_strings = []
coefficients = []
x_terms = []
z_terms = []
# 线性和二次项
for j in range(m):
    # constant_term = -1/2*(np.sum(p[:, j]) - ( 2 ** l-1))
    # h_terms = []

    for i in range(n):
        coef =-1/2 * (np.sum(p[:, j]) - ( 2 ** l-1)) * p[i, j] * P
        x_term = pauli_z_term(i * m + j, num_qubits)
        x_terms.append((coef, x_term))
        # pauli_strings.append(x_term)
        # cofficients.append(coef)

    for h in range(l):
        coef = -1/2 * (np.sum(p[:, j]) - ( 2 ** l-1)) * - (2 ** h) * P
        z_term = pauli_z_term(m * n + h, num_qubits)
        # pauli_strings.append(z_term)
        # cofficients.append(coef)
        x_terms.append((coef,z_term))

    for i1 in range(n):
        for i2 in range(n):
            if i1 != i2:
                coef = 1/4 * p[i1, j] * p[i2, j] * P
                x_term1 = pauli_z_term(i1 * m + j, num_qubits)
                x_term2 = pauli_z_term(i2 * m + j, num_qubits)
                combined_op = ''.join(['I' if c1 == 'I' and c2 == 'I' else 'Z' if c1 == 'Z' or c2 == 'Z' else 'I'
                                           for c1, c2 in zip(x_term1, x_term2)])
                # pauli_strings.append(combined_op)
                # cofficients.append(coef)
                x_terms.append((coef, combined_op))
    for i in range(n):
        for h in range(l):
            coef = 1/4 * -2 * p[i,j] * 2 ** h * P
            x_term = pauli_z_term(i * m + j, num_qubits)
            z_term = pauli_z_term(m * n + h, num_qubits)
            combined_op = ''.join(['I' if c1 == 'I' and c2 == 'I' else 'Z' if c1 == 'Z' or c2 == 'Z' else 'I'
                                           for c1, c2 in zip(x_term, z_term)])
            # pauli_strings.append(combined_op)
            # cofficients.append(coef)
            x_terms.append((coef, combined_op))
    for h1 in range(l):
        for h2 in range(l):
            if h1 != h2:
                coef = 1/4 * 2 ** h1 * 2 ** h2 * P
                z_term1 = pauli_z_term(m * n + h1, num_qubits)
                z_term2= pauli_z_term(m * n + h2, num_qubits)
                combined_op = ''.join(['I' if c1 == 'I' and c2 == 'I' else 'Z' if c1 == 'Z' or c2 == 'Z' else 'I'
                                           for c1, c2 in zip(z_term1, z_term2)])
                # pauli_strings.append(combined_op)
                # cofficients.append(coef)
                x_terms.append((coef, combined_op))
pauli_strings = [term[1] for term in x_terms]
coefficients = [term[0] for term in x_terms]
# 转换为 SparsePauliOp
paulis = [Pauli(label) for label in pauli_strings]
hamiltonian3 = SparsePauliOp.from_list(list(zip(pauli_strings, coefficients)))

print(hamiltonian3)

SparsePauliOp(['ZIIIIII', 'IIIZIII', 'IIIIIIZ', 'ZIIZIII', 'ZIIZIII', 'ZIIIIIZ', 'IIIZIIZ', 'IZIIIII', 'IIIIZII', 'IIIIIIZ', 'IZIIZII', 'IZIIZII', 'IZIIIIZ', 'IIIIZIZ', 'IIZIIII', 'IIIIIZI', 'IIIIIIZ', 'IIZIIZI', 'IIZIIZI', 'IIZIIIZ', 'IIIIIZZ'],
              coeffs=[ -30.+0.j,  -90.+0.j,   30.+0.j,   15.+0.j,   15.+0.j,  -10.+0.j,
  -30.+0.j,  -40.+0.j,  -20.+0.j,   20.+0.j,   10.+0.j,   10.+0.j,
  -20.+0.j,  -10.+0.j, -120.+0.j,  -80.+0.j,   40.+0.j,   30.+0.j,
   30.+0.j,  -30.+0.j,  -20.+0.j])


In [6]:
cost_hamiltonian = hamiltonian2
cost_hamiltonian_xy = hamiltonian1 + hamiltonian3
print(cost_hamiltonian)

SparsePauliOp(['ZIIIIII', 'IZIIIII', 'IIZIIII', 'IIIZIII', 'IIIIZII', 'IIIIIZI', 'ZZIIIII', 'ZIZIIII', 'ZZIIIII', 'IZZIIII', 'ZIZIIII', 'IZZIIII', 'IIIZZII', 'IIIZIZI', 'IIIZZII', 'IIIIZZI', 'IIIZIZI', 'IIIIZZI'],
              coeffs=[-20.+0.j, -20.+0.j, -20.+0.j, -20.+0.j, -20.+0.j, -20.+0.j,   5.+0.j,
   5.+0.j,   5.+0.j,   5.+0.j,   5.+0.j,   5.+0.j,   5.+0.j,   5.+0.j,
   5.+0.j,   5.+0.j,   5.+0.j,   5.+0.j])


In [7]:
# 初始化数组
pauli_strings = []
coefficients = []
x_terms = []
z_terms = []
for i in range(num_qubits):
    coef = 1
    x_term = pauli_x_term(i,num_qubits)
    x_terms.append((coef, x_term))

# 提取所有的 Pauli 字符串和系数
pauli_strings = [term[1] for term in  x_terms ]
coefficients = [term[0] for term in  x_terms ]

# 转换为 SparsePauliOp
paulis = [Pauli(label) for label in pauli_strings]
mixer_hamiltonian = SparsePauliOp.from_list(list(zip(pauli_strings, coefficients)))

print(mixer_hamiltonian)

SparsePauliOp(['XIIIIII', 'IXIIIII', 'IIXIIII', 'IIIXIII', 'IIIIXII', 'IIIIIXI', 'IIIIIIX'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j])


In [8]:
# 初始化数组
pauli_strings = []
coefficients = []
x_terms = []
z_terms = []
# for i in range(num_qubits):
#     coef = 1
#     x_term = pauli_x_term(i,num_qubits)
#     x_terms.append((coef, x_term))
# 设置前 n*m 个量子比特为 XY 混合器
xy_terms = []
for i in range(n):
    for j in range(m):
        # 确定量子比特对 (i * m + j) 和 (i * m + j + 1)
        if j < m - 1:
            coef = 0.5  # XY 项的系数

            # 生成 Pauli-X 和 Pauli-Y 操作的字符串
            x_term = 'I' * (i * m + j) + 'X' + 'I' * (num_qubits - i * m - j - 1)
            y_term = 'I' * (i * m + j) + 'Y' + 'I' * (num_qubits - i * m - j - 1)
            x_term_next = 'I' * (i * m + j + 1) + 'X' + 'I' * (num_qubits - i * m - j - 2)
            y_term_next = 'I' * (i * m + j + 1) + 'Y' + 'I' * (num_qubits - i * m - j - 2)

            # 组合成 XY 项
            xy_term_x = x_term[:i * m + j] + 'X' + x_term_next[i * m + j + 1:]
            xy_term_y = y_term[:i * m + j] + 'Y' + y_term_next[i * m + j + 1:]

            # 将生成的 XY 项添加到 xy_terms 中
            xy_terms.append((coef, xy_term_x))
            xy_terms.append((coef, xy_term_y))

# 提取所有的 Pauli 字符串和系数
pauli_strings = [term[1] for term in  xy_terms ]
coefficients = [term[0] for term in  xy_terms ]

# 转换为 SparsePauliOp
paulis = [Pauli(label) for label in pauli_strings]
mixer_hamiltonian_xy = SparsePauliOp.from_list(list(zip(pauli_strings, coefficients)))

print(mixer_hamiltonian_xy)

SparsePauliOp(['XXIIIII', 'YYIIIII', 'IXXIIII', 'IYYIIII', 'IIIXXII', 'IIIYYII', 'IIIIXXI', 'IIIIYYI'],
              coeffs=[0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j])


In [9]:
def invert_counts(s):
    return s[::-1]

In [10]:
from qiskit.circuit.library import PauliEvolutionGate
from qiskit import Aer, transpile, assemble, QuantumCircuit, execute


# 定义成本函数
def cost_function(params, n, m, l, K1, P, p):
    beta, gamma = params[:len(params)//2], params[len(params)//2:]
    num_qubits = n * m + l
    qc = QuantumCircuit(num_qubits, num_qubits)  # 添加经典寄存器

    # 初始化量子态到 |+> 态
    qc.h(range(n * m + l))

    for g, b in zip(gamma, beta):
        # 应用问题哈密顿量
        qc.append(PauliEvolutionGate(cost_hamiltonian, g), list(range(num_qubits)))
        # 应用混合哈密顿量
        qc.append(PauliEvolutionGate(mixer_hamiltonian, b), list(range(num_qubits)))

    # print(qc)
    # 模拟电路
    # 添加测量指令
    qc.measure(range(num_qubits), range(num_qubits))
    backend = Aer.get_backend('qasm_simulator')
    # qc = transpile(qc, backend)
    # qobj = assemble(qc, shots=10000)
    # result = backend.run(qobj).result()
    result = execute(qc, backend, seed_simulator=10, shots=10000).result()
    counts = result.get_counts(qc)
    # 计算期望值
    expectation = 0
    for bitstring, count in counts.items():
        bitstring = invert_counts(bitstring)
        prob = count / 10000
        z_h = np.array([int(bit) for bit in bitstring[(n*m):]])
        x_ij = np.array([int(bit) for bit in bitstring[:(n*m)]]).reshape(n, m)

        E1 = np.sum([2 ** h * z for h, z in enumerate(z_h)])
        E2 = K1 * np.sum([(np.sum(x_ij[i]) - 1) ** 2 for i in range(n)])
        # alpha_s = np.max([0, np.sum(x_ij * p) - np.sum([2 ** h * z for h, z in enumerate(z_h)])])
        E3 = 0
        for j in range(m):

            sum_xp = np.sum([x_ij[i][j] * p[i][j] for i in range(n)])

            sum_zh = np.sum([2**h * z_h[h] for h in range(l)])

            max_value = max(0, sum_xp - sum_zh)

            E3 += P * (max_value)**2
        E = E2
        expectation += E * prob
    return expectation, counts

In [11]:
from qiskit_algorithms.optimizers import COBYLA


# 优化 QAOA 参数
def optimize_qaoa(n, m, l, K1, P, p, params):
    def objective(params):
        expectation, _ = cost_function(params, n, m, l, K1, P, p)

        return expectation

    optimizer = COBYLA(rhobeg=1.5, tol=1e-7)
    # params = np.random.rand(2 * p_max) * 2 * np.pi
    result = optimizer.minimize(fun=objective, x0=params)

    return result

In [12]:
import time

# 运行优化
# p_max = 6  # 最大层数

startTime = time.time()
ansatz_final = object()
init_point = np.array([])
counts = {}
min_energy = 0
final_result = object()
for p1 in range(1, 4):
    min_energy = 10000
    for k in range(100):
        print('第：', p1, '层', ' ', '第', k, '个参数')
        init_point_temp = init_point
        beta_p = np.random.uniform(0, np.pi)
        gamma_p = np.random.uniform(0,  2 * np.pi)
        if k >0:
            init_point_temp = np.delete(init_point_temp, 2 * p1 - 1)
            init_point_temp = np.delete(init_point_temp, p1 - 1)
        init_point_temp = np.insert(init_point_temp, p1 - 1, beta_p)
        init_point_temp = np.insert(init_point_temp, 2 * p1 - 1, gamma_p)
        # 创建参数向量
        # print("init_point_temp"+ init_point_temp)
        result = optimize_qaoa(n, m, l, K1, P, p, init_point_temp)

        # print(result.x)
        optimal_params = result.x

        # 计算最终的能量
        energy, counts_temp = cost_function(result.x, n, m, l, K1, P, p)
        # 寻找这20对点中最优的参数
        if min_energy > energy:
            min_energy = energy
            init_point = optimal_params
            counts = counts_temp
            final_result = result
endTime = time.time()
# params = np.random.rand(2 * p_max,2*np.pi)
# result = optimize_qaoa(n, m, l, K1, P, p, params)

第： 1 层   第 0 个参数
第： 1 层   第 1 个参数
第： 1 层   第 2 个参数
第： 1 层   第 3 个参数
第： 1 层   第 4 个参数
第： 1 层   第 5 个参数
第： 1 层   第 6 个参数
第： 1 层   第 7 个参数
第： 1 层   第 8 个参数
第： 1 层   第 9 个参数
第： 1 层   第 10 个参数
第： 1 层   第 11 个参数
第： 1 层   第 12 个参数
第： 1 层   第 13 个参数
第： 1 层   第 14 个参数
第： 1 层   第 15 个参数
第： 1 层   第 16 个参数
第： 1 层   第 17 个参数
第： 1 层   第 18 个参数
第： 1 层   第 19 个参数
第： 1 层   第 20 个参数
第： 1 层   第 21 个参数
第： 1 层   第 22 个参数
第： 1 层   第 23 个参数
第： 1 层   第 24 个参数
第： 1 层   第 25 个参数
第： 1 层   第 26 个参数
第： 1 层   第 27 个参数
第： 1 层   第 28 个参数
第： 1 层   第 29 个参数
第： 1 层   第 30 个参数
第： 1 层   第 31 个参数
第： 1 层   第 32 个参数
第： 1 层   第 33 个参数
第： 1 层   第 34 个参数
第： 1 层   第 35 个参数
第： 1 层   第 36 个参数
第： 1 层   第 37 个参数
第： 1 层   第 38 个参数
第： 1 层   第 39 个参数
第： 1 层   第 40 个参数
第： 1 层   第 41 个参数
第： 1 层   第 42 个参数
第： 1 层   第 43 个参数
第： 1 层   第 44 个参数
第： 1 层   第 45 个参数
第： 1 层   第 46 个参数
第： 1 层   第 47 个参数
第： 1 层   第 48 个参数
第： 1 层   第 49 个参数
第： 1 层   第 50 个参数
第： 1 层   第 51 个参数
第： 1 层   第 52 个参数
第： 1 层   第 53 个参数
第： 1 层   第 54 个参数
第： 1 层   第 55 个参数
第：

In [13]:
# 输出结果
print("运行时间：",endTime - startTime)
print("Optimal parameters:", final_result.x)
print("Optimal value:", final_result.fun)
# 使用最优参数运行量子电路并输出测量结果
final_expectation, final_counts = cost_function(final_result.x, n, m, l, K1, P, p)
# sorted_dict = dict(sorted(final_counts.items(), key=lambda item: item[1], reverse=True))
sorted_dict = {item[0][::-1]: item[1] for item in sorted(final_counts.items(), key=lambda item: item[1], reverse=True)}
print("Final expectation value:", final_expectation)
print("Final measurement counts:", sorted_dict)

运行时间： 1286.5034379959106
Optimal parameters: [ 4.76773089 -0.63472136  2.12461239  3.39759693  7.38854319  4.17458329]
Optimal value: 18.982000000000003
Final expectation value: 18.982000000000003
Final measurement counts: {'0100001': 526, '1001001': 519, '0010010': 511, '1010010': 510, '1010001': 509, '0001001': 505, '1100010': 503, '0010001': 498, '1010100': 492, '1001100': 485, '0100100': 479, '0001100': 477, '0100010': 476, '1001010': 472, '0001010': 458, '1100100': 457, '0010100': 454, '1100001': 434, '1000001': 112, '1100000': 105, '0001000': 105, '0000001': 102, '0000100': 102, '1000100': 99, '1001000': 98, '0100000': 97, '1010000': 95, '0000010': 85, '1000010': 83, '0010000': 77, '0000000': 23, '1000000': 12, '0010110': 4, '1010101': 3, '1011001': 2, '0100011': 2, '0010011': 2, '0101001': 2, '0010111': 2, '0110010': 2, '0100101': 2, '1001110': 1, '0010101': 1, '0011001': 1, '0111000': 1, '1100110': 1, '0001110': 1, '0111001': 1, '0101000': 1, '0100111': 1, '0001101': 1, '011000

In [14]:
# 定义成本函数
def cost_function_xy(params, n, m, l, K1, P, p, optimal_params):
    beta, gamma = params[:len(params)//2], params[len(params)//2:]
    opt_beta, opt_gamma = optimal_params[:len(optimal_params)//2], optimal_params[len(optimal_params)//2:]
    num_qubits = n * m + l
    qc = QuantumCircuit(num_qubits, num_qubits)  # 添加经典寄存器

    # 初始化量子态到 |+> 态
    # qc.h(range(n * m + l))
    for g, b in zip(opt_gamma, opt_beta):
        # 应用问题哈密顿量
        qc.append(PauliEvolutionGate(cost_hamiltonian, g), list(range(num_qubits)))
        # 应用混合哈密顿量
        qc.append(PauliEvolutionGate(mixer_hamiltonian, b), list(range(num_qubits)))

    for g, b in zip(gamma, beta):
        # 应用问题哈密顿量
        qc.append(PauliEvolutionGate(cost_hamiltonian_xy, g), list(range(num_qubits)))
        # 应用混合哈密顿量
        qc.append(PauliEvolutionGate(mixer_hamiltonian_xy, b), list(range(num_qubits)))

    # print(qc)
    # 模拟电路
    # 添加测量指令
    qc.measure(range(num_qubits), range(num_qubits))
    backend = Aer.get_backend('qasm_simulator')
    # qc = transpile(qc, backend)
    # qobj = assemble(qc, shots=10000)
    # result = backend.run(qobj).result()
    result = execute(qc, backend, seed_simulator=10, shots=10000).result()
    counts = result.get_counts(qc)
    # 计算期望值
    expectation = 0
    for bitstring, count in counts.items():
        bitstring = invert_counts(bitstring)
        prob = count / 10000
        z_h = np.array([int(bit) for bit in bitstring[(n*m):]])
        x_ij = np.array([int(bit) for bit in bitstring[:(n*m)]]).reshape(n, m)

        E1 = np.sum([2 ** h * z for h, z in enumerate(z_h)])
        E2 = K1 * np.sum([(np.sum(x_ij[i]) - 1) ** 2 for i in range(n)])
        # alpha_s = np.max([0, np.sum(x_ij * p) - np.sum([2 ** h * z for h, z in enumerate(z_h)])])
        E3 = 0
        for j in range(m):

            sum_xp = np.sum([x_ij[i][j] * p[i][j] for i in range(n)])

            sum_zh = np.sum([2**h * z_h[h] for h in range(l)])

            max_value = max(0, sum_xp - sum_zh)

            E3 += P * (max_value)**2
        E = E1 + E3
        expectation += E * prob
    return expectation, counts

In [15]:
# 优化 QAOA 参数
def optimize_qaoa_xy(n, m, l, K1, P, p, params, opt_params):
    def objective(params):
        expectation, _ = cost_function_xy(params, n, m, l, K1, P, p, opt_params)

        return expectation

    optimizer = COBYLA(rhobeg=1.5, tol=1e-7)
    # params = np.random.rand(2 * p_max) * 2 * np.pi
    result = optimizer.minimize(fun=objective, x0=params)

    return result

In [16]:
import time

# 运行优化
# p_max = 6  # 最大层数

startTime = time.time()
# ansatz_final = object()
init_point_xy = np.array([])
counts_xy = {}
min_energy_xy = 0
final_result_xy = object()
optimal_params = final_result.x
for p1 in range(1, 4):
    min_energy_xy = 10000
    for k in range(200):
        print('第：', p1, '层', ' ', '第', k, '个参数')
        init_point_temp = init_point_xy
        beta_p = np.random.uniform(0, np.pi)
        gamma_p = np.random.uniform(0,  2 * np.pi)
        if k >0:
            init_point_temp = np.delete(init_point_temp, 2 * p1 - 1)
            init_point_temp = np.delete(init_point_temp, p1 - 1)
        init_point_temp = np.insert(init_point_temp, p1 - 1, beta_p)
        init_point_temp = np.insert(init_point_temp, 2 * p1 - 1, gamma_p)
        # 创建参数向量
        # print("init_point_temp"+ init_point_temp)
        result = optimize_qaoa_xy(n, m, l, K1, P, p, init_point_temp, optimal_params)

        # print(result.x)
        optimal_params_xy = result.x

        # 计算最终的能量
        energy, counts_temp = cost_function_xy(result.x, n, m, l, K1, P, p,optimal_params)
        # 寻找这20对点中最优的参数
        if min_energy_xy > energy:
            min_energy_xy = energy
            init_point_xy = optimal_params_xy
            counts_xy = counts_temp
            final_result_xy = result
endTime = time.time()

第： 1 层   第 0 个参数
第： 1 层   第 1 个参数
第： 1 层   第 2 个参数
第： 1 层   第 3 个参数
第： 1 层   第 4 个参数
第： 1 层   第 5 个参数
第： 1 层   第 6 个参数
第： 1 层   第 7 个参数
第： 1 层   第 8 个参数
第： 1 层   第 9 个参数
第： 1 层   第 10 个参数
第： 1 层   第 11 个参数
第： 1 层   第 12 个参数
第： 1 层   第 13 个参数
第： 1 层   第 14 个参数
第： 1 层   第 15 个参数
第： 1 层   第 16 个参数
第： 1 层   第 17 个参数
第： 1 层   第 18 个参数
第： 1 层   第 19 个参数
第： 1 层   第 20 个参数
第： 1 层   第 21 个参数
第： 1 层   第 22 个参数
第： 1 层   第 23 个参数
第： 1 层   第 24 个参数
第： 1 层   第 25 个参数
第： 1 层   第 26 个参数
第： 1 层   第 27 个参数
第： 1 层   第 28 个参数
第： 1 层   第 29 个参数
第： 1 层   第 30 个参数
第： 1 层   第 31 个参数
第： 1 层   第 32 个参数
第： 1 层   第 33 个参数
第： 1 层   第 34 个参数
第： 1 层   第 35 个参数
第： 1 层   第 36 个参数
第： 1 层   第 37 个参数
第： 1 层   第 38 个参数
第： 1 层   第 39 个参数
第： 1 层   第 40 个参数
第： 1 层   第 41 个参数
第： 1 层   第 42 个参数
第： 1 层   第 43 个参数
第： 1 层   第 44 个参数
第： 1 层   第 45 个参数
第： 1 层   第 46 个参数
第： 1 层   第 47 个参数
第： 1 层   第 48 个参数
第： 1 层   第 49 个参数
第： 1 层   第 50 个参数
第： 1 层   第 51 个参数
第： 1 层   第 52 个参数
第： 1 层   第 53 个参数
第： 1 层   第 54 个参数
第： 1 层   第 55 个参数
第：

In [17]:
# 输出结果
print("运行时间：",endTime - startTime)
print("Optimal parameters:", final_result_xy.x)
print("Optimal value:", final_result_xy.fun)
# 使用最优参数运行量子电路并输出测量结果
final_expectation_xy, final_counts_xy = cost_function_xy(final_result.x, n, m, l, K1, P, p, optimal_params)
# sorted_dict = dict(sorted(final_counts.items(), key=lambda item: item[1], reverse=True))
sorted_dict_xy = {item[0][::-1]: item[1] for item in sorted(final_counts_xy.items(), key=lambda item: item[1], reverse=True)}
print("Final expectation value:", final_expectation_xy)
print("Final measurement counts:", sorted_dict_xy)

运行时间： 5539.798525810242
Optimal parameters: [4.69153572 1.00157253 1.24505329 7.04161092 5.35742227 1.4035915 ]
Optimal value: 151.06220000000005
Final expectation value: 191.09140000000002
Final measurement counts: {'0000000': 1249, '0111000': 623, '0000111': 617, '0000010': 541, '0001000': 470, '0000011': 464, '0110000': 392, '0000101': 386, '0011000': 351, '0111101': 318, '0111111': 316, '0010111': 304, '0111001': 302, '0011111': 273, '0100000': 273, '0000100': 175, '0101011': 160, '0011001': 146, '0101111': 140, '0011101': 134, '0010101': 124, '0100011': 119, '0001100': 115, '0100010': 106, '0000001': 97, '0001101': 97, '0110110': 89, '0010100': 88, '0110100': 87, '0101110': 81, '0001010': 81, '0001110': 78, '0111110': 77, '0101010': 77, '0110001': 75, '0010001': 73, '0011110': 66, '0101000': 64, '0001011': 62, '0010000': 61, '0111010': 59, '0100111': 57, '0001111': 54, '0110010': 51, '0110101': 46, '0111011': 43, '0001001': 41, '0010011': 40, '0101100': 35, '0101101': 29, '0011100

In [ ]:
expectation = 0

bitstring = "10010"

z_h = np.array([int(bit) for bit in bitstring[(n*m):]])
x_ij = np.array([int(bit) for bit in bitstring[:(n*m)]]).reshape(n, m)

# alpha_s = np.max([0, np.sum(x_ij * p) - np.sum([2 ** h * z for h, z in enumerate(z_h)])])
E3 = 0
for j in range(m):
    # 计算 ∑_{i=1}^{n} x_{ij} p_{ij}
    sum_xp = np.sum([x_ij[i][j] * p[i][j] for i in range(n)])

    # 计算 ∑_{h=0}^{l-1} 2^h * z_h
    sum_zh = np.sum([2**h * z_h[h] for h in range(l)])

    # 计算 max{0, sum_xp - sum_zh}
    max_value = max(0, sum_xp - sum_zh)

    # 累加 P * (max_value)^2
    E3 += P * (max_value)**2
# E =  np.sum([2 ** h * z for h, z in enumerate(z_h)]) + K1 * np.sum([(np.sum(x_ij[i]) - 1) ** 2 for i in range(n)]) + E3
E0 = np.sum([2 ** h * z for h, z in enumerate(z_h)])
E1 = K1 * np.sum([(np.sum(x_ij[i]) - 1) ** 2 for i in range(n)])
E =  E0 + E1 + E3
# expectation += E * prob
print(E0)
print(E1)
print(E3)
print(E)